# Demonstration

The following demonstration will use the training set of the OHSUMED corpus. This training set was used in the Filtering Track of the 9th edition of the Text REtrieval Conference (TREC-9). We will use it for the information retrieval exercises of this workshop. Download and unzip [ohsumed.zip] into the same folder as this notebook. 

To help you read the data, we are providing the file ohsumed.py (in the zip file above) that has a simple API to the data. When you import it at the Python prompt, it will provide the following variables:


1. `index`: a dictionary with document IDs as keys, and document text as values.
2. `questions`: a dictionary with query IDs as keys, and query text as values.
3. `answers`: a dictionary with query IDs as keys, and a set with the IDs of known relevant documents for evaluation purposes.


In [1]:
import ohsumed

Reading OHSUMED data


In [2]:
len(ohsumed.index)

54710

In [3]:
len(ohsumed.questions)

63

In [4]:
len(ohsumed.answers)

63

## Inverted index

We are going to build an inverted index of the non-stop words with frequency higher than 5.

The following code reads the files and creates a counter of all words in the corpus. We will use NLTK's word tokeniser (read the beginning of [chapter 3 of NLTK's book](http://www.nltk.org/book/ch03.html#processing-raw-text)) to convert each document into a list of tokens.

In [5]:
import nltk, collections
stop = nltk.corpus.stopwords.words('english')
wordcounter = collections.Counter([w.lower() for k in ohsumed.index
                                             for w in nltk.word_tokenize(ohsumed.index[k])])

The following code creates the inverted index of all non-stop words with frequency higher than 5.

In [7]:
inverted = dict()
for d in ohsumed.index:
    for w in nltk.word_tokenize(ohsumed.index[d]):
        w = w.lower()
        if w in stop or wordcounter[w] <= 5:
            continue
        if w in inverted:
            inverted[w].add(d)
        else:
            inverted[w] = set([d])

In [12]:
list(inverted.keys())[:10]

['stroke-prone',
 'uncomfortable',
 'll',
 'microseconds',
 'fg',
 'neomucosal',
 'b-dna',
 't3-t6-',
 'breastfeeding',
 'drug-resistant']

In [13]:
inverted['stroke-prone']

{'87124462',
 '87179236',
 '87193011',
 '87249065',
 '87249082',
 '87249085',
 '87249096',
 '87264613',
 '87306829'}

The following code saves the inverted index into a pickle file. This way we do not need to recompute the inverted index again. Read [Python's documentation on pickle files](https://docs.python.org/3/library/pickle.html) for more detail. Note that the file we created is opened for writing in binary mode, following the advice of a [stackoverflow post about saving pickle files](http://stackoverflow.com/questions/13906623/using-pickle-dump-typeerror-must-be-str-not-bytes).

In [10]:
import pickle
with open('inverted.pickle','wb') as f:
    pickle.dump(inverted,f)

## Boolean retrieval

The following code reads the pickled file and returns the list of documents that maches this Boolean query:
1. (menopausal OR pregnant) AND woman AND NOT healthy

In [16]:
with open('inverted.pickle','rb') as f:
    inverted = pickle.load(f)

In [17]:
(inverted['menopausal'] | inverted['pregnant']) & inverted['woman'] - inverted['healthy']

{'87060673',
 '87066899',
 '87097274',
 '87097518',
 '87099263',
 '87114245',
 '87117852',
 '87128881',
 '87134330',
 '87138205',
 '87153548',
 '87153568',
 '87169457',
 '87185313',
 '87226668',
 '87231479',
 '87235637',
 '87251241',
 '87252385',
 '87261426',
 '87281235',
 '87290433',
 '87296136',
 '87316210',
 '87316220',
 '87316328',
 '87324028',
 '87325497'}

The following code builds an inverted index using the stems, not the whole words. Before stemming, all stop words are removed. After stemming, all words with frequency higher than 5 are ignored. For stemming the words, we will use the [Porter stemmer provided by NLTK](http://nltk.org/book/ch03#stemmers).

In [18]:
porter = nltk.stem.porter.PorterStemmer()
stemcounter = collections.Counter([porter.stem(w.lower()) for k in ohsumed.index
                                   for w in nltk.word_tokenize(ohsumed.index[k])])

In [19]:
stem_inverted = dict()
for d in ohsumed.index.keys():
    for w in nltk.word_tokenize(ohsumed.index[d]):
        w = w.lower()
        stem = porter.stem(w)
        if w in stop or stemcounter[stem] <= 5:
            continue
        if stem in stem_inverted:
            stem_inverted[stem].add(d)
        else:
            stem_inverted[stem] = set([d])

In [20]:
len(inverted)

30128

In [21]:
len(stem_inverted)

23684

Here is the same Boolean query on this new inverted index. Compare the results.

In [22]:
(stem_inverted[porter.stem('menopausal')] | stem_inverted[porter.stem('pregnant')]) & \
 stem_inverted[porter.stem('woman')] - stem_inverted[porter.stem('healthy')]

{'87054746',
 '87060673',
 '87066899',
 '87097274',
 '87097518',
 '87099263',
 '87114238',
 '87114241',
 '87114243',
 '87114245',
 '87114246',
 '87117852',
 '87128881',
 '87134330',
 '87138205',
 '87153548',
 '87153568',
 '87169457',
 '87185313',
 '87189233',
 '87210461',
 '87226668',
 '87231479',
 '87235637',
 '87251241',
 '87252385',
 '87261426',
 '87281235',
 '87290433',
 '87296136',
 '87299569',
 '87316210',
 '87316220',
 '87316328',
 '87324028',
 '87325497'}

# Your Turn

## Vector Retrieval

### Exercise: Boolean Information Retrieval

Create an inverted index of the NLTK Gutenberg corpus and save it into a file "gutenbergindex.pickle". To create this index there is no need to look for stop words or word frequencies, since the corpus is not that large. Simply use all the words. Use this index to find the documents that match the following Boolean queries:

1. Brutus OR Caesar
2. Brutus AND NOT Caesar
3. (Brutus AND Caesar) OR Calpurnia


### Exercise: tf.idf

Using scikit-learn, compute the tf.idf of all words in the OHSUMED corpus. Use the English list of stop words, and all other settings should use the default values. In particular, do not stem the words. Pickle the resulting tf.idf vectoriser into a file tfidf.pickle.

### Exercise: Sort by tf.idf

Write a program that prints the words of a document with highest tf.idf score, sorted in descending order.

### Optional exercise: frequency vs. tf.idf

Sort the words by frequency and plot their tf.idf score (we have seen this in the lectures). Can you see any interesting patterns in the plot?

### Exercise: tf.idf cosine similarity

Write a function that takes as a parameter a string and an optional parameter n the number of results, and returns the IDs of the n documents that are most relevant according to the tf.idf measure and the cosine similarity. The results are sorted in descending order of the cosine similarity score. 

In [10]:
def best_documents(querystring,n=10):
    "Return the best documents sorted by relevance"
    pass

## PageRank

The following graph represents a tiny collection of HTML documents and how they are linked. You will implement PageRank on the documents.

![pagerank](graph.gif)

### Exercise: Transition matrix

Write the transition matrix using Numpy Python code.

### Exercise: Iteration of PageRank

Perform an iteration of the PageRank algorithm and report on the resulting PageRank scores of each page.

### Exercise: Complete PageRank

Perform the PageRank algorithm until convergence and report the number of iterations and the final scores.